# Calculate angles and velocities 

Eucledian_distance.rename(columns={"Continuous_Time": "time", 
                                   "eyePositionCombinedWorld.x": "xcoord_orig", 
                                   "eyePositionCombinedWorld.z": "zcoord_orig", 
                                   "eyePositionCombinedWorld.y": "ycoord_orig", 
                                   "hitPointOnObject_x": "xhpoo", 
                                   "hitPointOnObject_y": "yhpoo", 
                                   "hitPointOnObject_z": "zhpoo" })

In [1]:
import glob
import re
import pandas as pd
import numpy as np 
import math 
from collections import Counter 
import seaborn as sns
import matplotlib.pyplot as plt 

In [2]:
# Since the collider names are too detailed, here we create a dictionary with patterns to classify them into our categories of interest
patterns = {'\d{2}_Sa':'Passive_Agent', '\d{2}_Cma':'Active_Agent', 'Building_\d+': 'Building'}
patterns.update(dict.fromkeys(['Castle-TaskBuilding_56','HighSilo-TaskBuilding_49', 'Windmill-TaskBuilding_10_1', 'Church_TaskBuilding_16'], 'Global_Landmark'))
patterns.update(dict.fromkeys(['TaskBuilding_2','TaskBuilding_3', 'TaskBuilding_5', 'TaskBuilding_8', 'TaskBuilding_9', 'TaskBuilding_11', 'TaskBuilding_13', 'TaskBuilding_14', 'TaskBuilding_20', 'TaskBuilding_21', 'TaskBuilding_23','TaskBuilding_27', 'TaskBuilding_29', 'TaskBuilding_32', 'TaskBuilding_34',  'TaskBuilding_38', 'TaskBuilding_41', 'TaskBuilding_42', 'TaskBuilding_44', 'TaskBuilding_45', 'TaskBuilding_47', 'TaskBuilding_50', 'TaskBuilding_51', 'TaskBuilding_52', 'BasketballCourt_58', 'Construction_57', 'Graffity_02', 'Graffity_03', 'Graffity_05', 'Graffity_08', 'Graffity_09', 'Graffity_11', 'Graffity_13', 'Graffity_14', 'Graffity_20', 'Graffity_21', 'Graffity_23', 'Graffity_27', 'Graffity_29', 'Graffity_32', 'Graffity_34', 'Graffity_38', 'Graffity_41', 'Graffity_42', 'Graffity_44', 'Graffity_45', 'Graffity_47',  'Graffity_50', 'Graffity_51', 'Graffity_52'], 'TaskBuilding_Public'))
patterns.update(dict.fromkeys(['TaskBuilding_1','TaskBuilding_4', 'TaskBuilding_6', 'TaskBuilding_7', 'TaskBuilding_12', 'TaskBuilding_15', 'TaskBuilding_17', 'TaskBuilding_18', 'TaskBuilding_19', 'TaskBuilding_22', 'TaskBuilding_24','TaskBuilding_25', 'TaskBuilding_26', 'TaskBuilding_28', 'TaskBuilding_30',  'TaskBuilding_31', 'TaskBuilding_33', 'TaskBuilding_35', 'TaskBuilding_36', 'TaskBuilding_37', 'TaskBuilding_39', 'TaskBuilding_40', 'TaskBuilding_43', 'TaskBuilding_48', 'TaskBuilding_54','TaskBuilding_55','Graffity_1','Graffity_4', 'Graffity_6', 'Graffity_7', 'Graffity_12', 'Graffity_15', 'Graffity_17', 'Graffity_18', 'Graffity_19', 'Graffity_22', 'Graffity_24','Graffity_25', 'Graffity_26', 'Graffity_28', 'Graffity_30',  'Graffity_31', 'Graffity_33', 'Graffity_35', 'Graffity_36', 'Graffity_37', 'Graffity_39', 'Graffity_40', 'Graffity_43', 'Graffity_48', 'Graffity_54', 'Graffity_55' ], 'TaskBuilding_Residential'))
default_val = 'Background'

In [3]:
# Function: calculate MAD saccade
def at_mad(angular_vel, th_0=200):
    # defines the saccade threshold (code from Ashima)
    threshs = []
    while True:
        # take th_0
        threshs.append(th_0)
        # get all angles smaller than this
        angular_vel = angular_vel[angular_vel < th_0]

        # MAD:
        # take the median of all angles smaller than th_0
        median = np.median(angular_vel)
        # substract the median value
        diff = np.sqrt((angular_vel - median) ** 2)
        # get the median of these values
        med_abs_deviation = np.median(diff)

        # calcualte the next threshold with the median
        # 1.486 used when assuming a normal distribution
        th_1 = median + 3 * 1.486 * med_abs_deviation
        # if the thresholds are too different, redo the while loop
        if abs(th_0 - th_1) > 1:
            th_0 = th_1
        # else, set the final threshold to the current one, break the while loop and return values
        else:
            saccade_thresh = th_1
            threshs.append(saccade_thresh)
            break
    return saccade_thresh, threshs

# Starts Here
Continuos colliders method

In [ ]:
path = "/Volumes/SSD/00_Data_Processing/Pre_processed/04_Interpolated" 

# csv files in the path
files = glob.glob(path + "/*.csv")
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    One_participant = pd.read_csv(filename)
    # Here we identify the shifts on collider name
    One_participant['Collider_shift'] = One_participant['Interpolated_collider'].shift(1) != One_participant['Interpolated_collider']
    # Create calculate a cumulative sum of the collider changes
    One_participant['counter'] = (One_participant['Collider_shift'] == True).cumsum()
    # Shift the counter column by one row to align it with the correct row
    One_participant['counter'] = One_participant['counter'].shift(1).fillna(0)
    One_participantC = One_participant.copy()
    #Create subset that only has the rows with shifts in colliders 
    One_participant_true = One_participant[One_participant['Collider_shift'] == True].reset_index().copy()
    # Since shift converts index into float we change it back into int so that it can be read as index
    One_participant_true["index_shift"] = One_participant_true["index"].shift(-1).astype('Int64')
    # Calculate the difference in time between each shift
    One_participant_true["Time_diff"] = One_participant_true.timeStampDataPointEnd.diff(1).shift(-1)
    One_participant_true.dropna(inplace=True)
    #### Create the gaze column 
    One_participant["Time_of_Gaze"] = np.nan
    low = One_participant_true["index"].to_list()
    up = One_participant_true["index_shift"].to_list()
    time =  One_participant_true["Time_diff"].to_list()
    ranges = list(zip(low, up))
    for i, (lower, upper) in enumerate(ranges):
        One_participant.loc[lower:upper,"Time_of_Gaze"]  = time[i]
    One_participant["Gaze"] = np.where(One_participant["Time_of_Gaze"] > .250, "Gaze", "Movement")
    One_participant_true["Gaze"] = np.where(One_participant_true["Time_diff"] > .250, "Gaze", "Movement")
    low = []
    up = []
    
    ######## Debbies Algorithm ########
    
    for_eye = One_participant.copy()
    time = for_eye["timeStampDataPointEnd"].tolist()
    
    ## Calculate Angular velocities
    # get individual coordinates
    subj = list(zip( for_eye["eyePositionCombinedWorld.x"],for_eye["eyePositionCombinedWorld.y"],for_eye["eyePositionCombinedWorld.z"]))
    hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))
    # v_gaze_vec: get difference in hpoo
    v_gaze_vec = list(zip(for_eye["hitPointOnObject_x"].diff(), for_eye["hitPointOnObject_y"].diff(),for_eye["hitPointOnObject_z"].diff()))
    # get difference in time:
    ts = for_eye["timeStampDataPointEnd"].diff().tolist()
    # gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
    g_vec = list(np.subtract(hpoo, subj))
    gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]
    # v_gaze_inplane: is a scalar indicating the velocity in world coordinates at the location that is gazed at orthogonal to the gaze axis.
    z1 = [np.dot(v_gaze_vec_i, gaze_vec_i) for v_gaze_vec_i, gaze_vec_i in zip(v_gaze_vec, gaze_vec)]
    # z = (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))
    z = [z1[element] * np.array(gaze_vec[element]) for element in range(len(z1))]
    # ||v_gaze_vec(t) - z||
    v_gaze_inplane = np.linalg.norm(np.array(v_gaze_vec) - z, axis=-1)
    #Eucledian distance between eye coordinates and hit on object
    sub_hpoo = np.linalg.norm(np.array(subj) - np.array(hpoo), axis=-1)
    # arctan2(v_gaze_inplane, sub_hpoo)
    w_gaze = np.arctan2(v_gaze_inplane, sub_hpoo).tolist()
    # Turn angle of radians into degrees over seconds 
    w_gaze = [(w / ts[idx] * 180 / math.pi) for idx, w in enumerate(w_gaze)]
    # save df
    for_eye["combined_vel"] = w_gaze
   
    ### 10 second for threshold calculation starts here
    
    int_len = 10  # number of seconds of the interval
    time = for_eye["timeStampDataPointEnd"].values
    start = [time[0]]
    end = []
    start_idx = [0]
    end_idx = []
    for t, ti in enumerate(time[1:], start=1):
        if ti - start[-1] > int_len:
            end.append(time[t - 1])
            end_idx.append(t - 1)
            start.append(ti)
            start_idx.append(t)
    # add the last timepoint to end
    end.append(time[-1])
    end_idx.append(len(time))

    # save it as new df
    int_data = pd.DataFrame({
        "start": start,
        "end": end,
        "start_idx": start_idx,
        "end_idx": end_idx
    })
    
    combined_vel = for_eye["combined_vel"]

    # to add the final thresholds to for each segement
    scct = []
    for s, srt in enumerate(int_data["start"].tolist()):
        # get the slice of the combined velocity
        angular_vel = combined_vel[start_idx[s] : end_idx[s]]
        # use the at_mad function to caluclate the threshold
        saccade_th, thres = at_mad(angular_vel)
        if np.isnan(saccade_th):
            scct.append(thres[0])
        else:
            # add it to scct
            scct.append(saccade_th)

    # add it to int_data and save
    int_data["thresh"] = scct
    thr = int_data["thresh"]

    # go through all time intervals
    thresh = [0.0] * len(time)
    
    for s, srt in enumerate(int_data["start"].tolist()):
        # repeat the threshold as often as the time interval is long
        thresh = (
            thresh[: start_idx[s]]
            + [thr[s]] * len(time[start_idx[s] : end_idx[s]])
            + thresh[end_idx[s]:]
        )

    # add the two lists (ht & et) to for_eye df
    for_eye["thresh"] = thresh

    # eye-tracking
    thres = int_data["thresh"].values
    combined_vel = for_eye["combined_vel"].values

    # define list where the fixations will be added too
    is_fix = [np.nan] * len(combined_vel)
    
    start = int_data["start_idx"].tolist()
    end = int_data["end_idx"].tolist()

    # eye-tracking
    thres = int_data["thresh"].tolist()
    combined_vel = for_eye["combined_vel"].tolist()


    for i in range(len(start)):
        av = combined_vel[start[i] : end[i]]
        # go through combined velocity and save all that are bigger than the threshold
        fix = [ti if ti < thres[i] else np.nan for ti in av]
        is_fix[start[i] : end[i]] = fix
    for_eye["isFix"] = is_fix

    # save data
    for_eye = pd.DataFrame(for_eye)
    
    #Everywhere  where there is Nans that is a saccade meaning this are the cells that are really fast 
    for_eye.reset_index(inplace=True)
    
    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    time = for_eye.timeStampDataPointEnd.tolist()
    index = for_eye.index.tolist()  # index of df for easier use

    start_time = time[0]  # update for each change
    start_idx = index[0]  # will be updated each event and used to add to the lists

    # to save:
    isFix = []
    combined_vel = []

    # if the first sample does not have any data
    if math.isnan(for_eye.iloc[0]["combined_vel"]) and not math.isnan(
        for_eye.iloc[1]["combined_vel"]
    ):
        start_time = time[1]   # update for each change
        start_idx = index[1]  # will be updated each event and used to add to the lists
        isFix = [np.nan]
        combined_vel = [np.nan]

    # starting with a sacc
    if math.isnan(for_eye.loc[start_idx]["isFix"]):
        event = 0  # == sacc
    # starting with a gaze
    else:
        event = 1  # == gaze

    # go through the list:
    for idx in index[index.index(start_idx) : -1]:
        curr_line = for_eye.loc[idx]
        next_line = for_eye.loc[idx+1]

        # gaze (--> sacc): now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
            if event == 1 and next_line.timeStampDataPointEnd - start_time < min_gaze_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event to small and we are in big saccade, change isFix, change combined_vel
            elif event == 0 and next_line.timeStampDataPointEnd - start_time < min_gaze_dur:
                isFix = isFix + [np.nan] * (idx + 1 - start_idx)
                combined_vel = combined_vel + [np.nan] * (idx + 1 - start_idx)
            # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
            elif next_line.timeStampDataPointEnd - start_time >= min_gaze_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 1  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = for_eye.loc[idx + 1]["timeStampDataPointEnd"]

        # sacc (--> gaze): now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
            if event == 0 and next_line.timeStampDataPointEnd - start_time < min_sacc_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event to small but we are in big gaze, change isFix, change combined_vel
            elif event == 1 and next_line.timeStampDataPointEnd - start_time < min_sacc_dur:
                isFix = (
                    isFix
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # change isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
            elif next_line.timeStampDataPointEnd - start_time >= min_sacc_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 0  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = for_eye.loc[idx + 1]["timeStampDataPointEnd"]

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
                if (
                    event == 1
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small and we are in big saccade, change isFix, change combined_vel
                elif (
                    event == 0
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = isFix + [np.nan] * (idx + 2 - start_idx)
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )
                # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
                elif next_line.timeStampDataPointEnd + 0.011 - start_time >= min_gaze_dur:
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel
            # sacc:
            elif math.isnan(next_line.isFix):
                # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
                if (
                    event == 0
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small but we are in big gaze, change isFix, change combined_vel
                elif (
                    event == 1
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # change isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
                elif next_line.timeStampDataPointEnd + 0.011 - start_time >= min_sacc_dur:
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel

    # save everything:
    for_eye["isFix"] = isFix
    for_eye["corrected_vel"] = combined_vel
    # save data
    for_eye = pd.DataFrame(for_eye)
    
    time = for_eye.timeStampDataPointEnd.tolist()
    
    ########## EVENTS, LENGTH, AVG DISTANCE, NAME OF OBJECT ##########
    index = for_eye.index.tolist()  # index of df for easier use

    events = [np.nan] * len(
        for_eye
    )  # sacc begin == 1, sacc end == -1; gaze begin == 2, gaze end == -2

    # if the first sample does not have any data
    if math.isnan(for_eye.iloc[0]["combined_vel"]) and not math.isnan(
        for_eye.iloc[1]["combined_vel"]
    ):
        start_idx = index[
            1
        ]  # will be updated each event and used to add to the lists
        events[1] = 2
        length = [np.nan]
        dist = [
            np.nan
        ]  # to save the distance to the hitpoint at each timestamps
        avg_dist = [
            np.nan
        ]  # to save the average distance of collider(s) during event
        names = [np.nan]  # to save the name of the current gaze
    else:
        start_idx = index[
            0
        ]  # will be updated each event and used to add to the lists
        length = []
        dist = []  # to save the distance to the hitpoint at each timestamps
        avg_dist = (
            []
        )  # to save the average distance of collider(s) during event
        names = []  # to save the name of the current gaze
        if math.isnan(for_eye.iloc[index[0]]["combined_vel"]):
            events[0] = 1
        else:
            events[0] = 2

    start_time = for_eye.loc[start_idx]["timeStampDataPointEnd"].tolist()
    # go through the list:
    for idx in index[index.index(start_idx) : -1]:
        curr_line = for_eye.loc[idx]
        next_line = for_eye.loc[idx + 1]

        # distance:
        hpoo = np.array(
            [curr_line.hitPointOnObject_x, curr_line.hitPointOnObject_y, curr_line.hitPointOnObject_z]
        )  # hitpoints on object
        coord_orig = np.array(
            [
                curr_line["eyePositionCombinedWorld.x"],
                curr_line["eyePositionCombinedWorld.y"],
                curr_line["eyePositionCombinedWorld.z"],
            ]
        )  # position of eyes
        dist = dist + [
            np.linalg.norm(hpoo - coord_orig)
        ]  # calculate to distance at this timpoint

        # gaze --> sacc: now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            # get name:
            res = dict(
                Counter(for_eye.loc[start_idx:idx, "Interpolated_collider"].values.tolist())
            )
            names = names + [
                max(res.keys(), key=(lambda new_k: res[new_k]))
            ] * (idx + 1 - start_idx)
            # length, distance, events
            length = length + [curr_line.timeStampDataPointEnd - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event
            events[index.index(idx)] = -2  # end of gaze
            events[index.index(idx) + 1] = 1  # beginning of sacc
            # new idx
            start_time = curr_line.timeStampDataPointEnd
            start_idx = idx + 1

        # sacc --> gaze: now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            # get name:
            res = dict(
                Counter(for_eye.loc[start_idx:idx, "Interpolated_collider"].values.tolist())
            )
            names = names + [
                max(res.keys(), key=(lambda new_k: res[new_k]))
            ] * (idx + 1 - start_idx)
            # length, distance, events
            length = length + [curr_line.timeStampDataPointEnd - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event
            events[index.index(idx)] = -1  # end of sacc
            events[index.index(idx) + 1] = 2  # beginning of gaze
            # new idx
            start_time = curr_line.timeStampDataPointEnd
            start_idx = idx + 1

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                events[-1] = -2  # end of gaze
            # sacc:
            elif math.isnan(next_line.isFix):
                events[-1] = -1  # end of sacc
            length = length + [next_line.timeStampDataPointEnd - start_time] * (
                idx + 2 - start_idx
            )  # length of event
            # distance
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 2 - start_idx
            )  # average distance to collider(s) during event
            hpoo = np.array(
                [curr_line.hitPointOnObject_x, curr_line.hitPointOnObject_y, curr_line.hitPointOnObject_z]
            )  # hitpoints on object
            coord_orig = np.array(
                [
                    curr_line["eyePositionCombinedWorld.x"],
                    curr_line["eyePositionCombinedWorld.y"],
                    curr_line["eyePositionCombinedWorld.z"],
                ]
            )  # position of eyes
            dist = dist + [
                np.linalg.norm(hpoo - coord_orig)
            ]  # calculate to distance at this timpoint
            # names
            res = dict(
                Counter(for_eye.loc[start_idx:, "Interpolated_collider"].values.tolist())
            )
            names = names + [
                max(res.keys(), key=(lambda new_k: res[new_k]))
            ] * (idx + 2 - start_idx)
    # save everything:
    for_eye["events"] = events
    for_eye["length"] = length
    for_eye["distance"] = dist
    for_eye["avg_dist"] = avg_dist
    for_eye["names"] = names
    # display(for_eye[['time','isFix','events','hon_all','names']])
    # save data
    for_eye = pd.DataFrame(for_eye)
    # Change average distance to correct for the potential of other events 
    # so distance and avg_dist

    # total lists:
    all_dist = []
    avg_dist = []

    # updated after each gaze
    dist = []
    hon_pos = []
    dur_gaze = False

    # during event:
    # go through the list:      
    for g,gz in enumerate(for_eye['events']):
        curr_line = for_eye.loc[g]
        if gz == 2.0 or gz == 1.0:
            dur_gaze = True
            # get the gazed at object
            curr_gaze = curr_line.names
        # if you are currently in a gaze:
        if dur_gaze:
            # if you are currently having the correct element, add the position
            if curr_line.Interpolated_collider == curr_gaze:    
                hon_pos = hon_pos + [[curr_line.hitPointOnObject_x,
                    curr_line.hitPointOnObject_y,
                    curr_line.hitPointOnObject_z,]]
            dist = dist + [np.array([curr_line["eyePositionCombinedWorld.x"],
                curr_line["eyePositionCombinedWorld.y"],
                curr_line["eyePositionCombinedWorld.z"],])]

        # once the gaze is over, take the avg_dist
        if gz == -2.0 or gz == -1.0:
            hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
            # calculate to distance at this timpoint
            dist = [np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))]
            all_dist = all_dist + dist
            # average distance during the gaze event
            avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

            # reset everything:
            dist = []
            hon_pos = []
            dur_gaze = False

        # if there are parts that are neither gaze nor saccade:
        if (not dur_gaze) and (gz not in [2.0,1.0]) and (len(all_dist) + len(dist) != g + 1):
            all_dist = all_dist + [np.nan]
            avg_dist = avg_dist + [np.nan]

        if len(all_dist) + len(dist) != g + 1:
            display(g)

    if dur_gaze:
        hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
        # calculate to distance at this timpoint
        dist = [np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))]
        all_dist = all_dist + dist
        # average distance during the gaze event
        avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

    # add them to for_eye
    for_eye['distance'] = all_dist
    for_eye['avg_dist'] = avg_dist
    
    for_eye["names"] =  for_eye.names.fillna(method='ffill').fillna(method='bfill')
    for_eye['Collider_CategoricalN'] = for_eye['names'].apply(lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))
    

    # save data
    for_eye = pd.DataFrame(for_eye)
    
    # Define the condition and the string to add
    Mask_1f = ((for_eye['Collider_CategoricalN'] == "Active_Agent") & (for_eye['Face_Hits'] == "Face")) | ((for_eye['Collider_CategoricalN'] == "Passive_Agent") & (for_eye['Face_Hits'] == "Face"))
    Mask_2f  = ((for_eye['Collider_Categorical'] == "Active_Agent") | (for_eye['Collider_Categorical'] == "Passive_Agent")) & (for_eye['Face_Hits'] == "Face")    
    string_to_add = "_Face"
    # Use the loc method to index the rows where the condition is met
    for_eye.loc[Mask_1f, 'Collider_CategoricalN'] = for_eye.loc[Mask_1f, 'Collider_CategoricalN'] + string_to_add
    for_eye.loc[Mask_2f, 'Collider_Categorical'] = for_eye.loc[Mask_2f, 'Collider_Categorical'] + string_to_add
    for_eye.to_csv(f"/Volumes/SSD/00_Data_Processing/Pre_processed/05_Debbies_gaze/{filename[-10:-4]}.csv", index=True)
    print(filename[-10:-4])

<ipython-input-4-535225a1d365>:532: RuntimeWarning: Mean of empty slice
  hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
<ipython-input-4-535225a1d365>:537: RuntimeWarning: Mean of empty slice
  avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)


0479_2


<ipython-input-4-535225a1d365>:532: RuntimeWarning: Mean of empty slice
  hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
<ipython-input-4-535225a1d365>:537: RuntimeWarning: Mean of empty slice
  avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)


0479_3


<ipython-input-4-535225a1d365>:532: RuntimeWarning: Mean of empty slice
  hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
<ipython-input-4-535225a1d365>:537: RuntimeWarning: Mean of empty slice
  avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)


...

3310_5


<ipython-input-4-535225a1d365>:532: RuntimeWarning: Mean of empty slice
  hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
<ipython-input-4-535225a1d365>:537: RuntimeWarning: Mean of empty slice
  avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)


3572_1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-535225a1d365>", line 190, in <module>
    curr_line = for_eye.loc[idx]
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 895, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 1124, in _getitem_axis
    return self._get_label(key, axis=axis)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py", line 1073, in _get_label
    return self.obj.xs(label, axis=axis)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py", line 3727, in xs
    index = self.index
KeyboardIn

In [4]:
path = "/Volumes/SSD/00_Data_Processing/Pre_processed/04_Interpolated" 

# csv files in the path
files = glob.glob(path + "/*.csv")
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    One_participant = pd.read_csv(filename)
    # Here we identify the shifts on collider name
    One_participant['Collider_shift'] = One_participant['Interpolated_collider'].shift(1) != One_participant['Interpolated_collider']
    # Create calculate a cumulative sum of the collider changes
    One_participant['counter'] = (One_participant['Collider_shift'] == True).cumsum()
    # Shift the counter column by one row to align it with the correct row
    One_participant['counter'] = One_participant['counter'].shift(1).fillna(0)
    One_participantC = One_participant.copy()
    #Create subset that only has the rows with shifts in colliders 
    One_participant_true = One_participant[One_participant['Collider_shift'] == True].reset_index().copy()
    # Since shift converts index into float we change it back into int so that it can be read as index
    One_participant_true["index_shift"] = One_participant_true["index"].shift(-1).astype('Int64')
    # Calculate the difference in time between each shift
    One_participant_true["Time_diff"] = One_participant_true.timeStampDataPointEnd.diff(1).shift(-1)
    One_participant_true.dropna(inplace=True)
    #### Create the gaze column 
    One_participant["Time_of_Gaze"] = np.nan
    low = One_participant_true["index"].to_list()
    up = One_participant_true["index_shift"].to_list()
    time =  One_participant_true["Time_diff"].to_list()
    ranges = list(zip(low, up))
    for i, (lower, upper) in enumerate(ranges):
        One_participant.loc[lower:upper,"Time_of_Gaze"]  = time[i]
    One_participant["Gaze"] = np.where(One_participant["Time_of_Gaze"] > .250, "Gaze", "Movement")
    One_participant_true["Gaze"] = np.where(One_participant_true["Time_diff"] > .250, "Gaze", "Movement")
    low = []
    up = []
    
    ######## Debbies Algorithm ########
    
    for_eye = One_participant.copy()
   
    time = for_eye["timeStampDataPointEnd"].tolist()
    # get individual coordinates
    subj = list(zip( for_eye["eyePositionCombinedWorld.x"],for_eye["eyePositionCombinedWorld.y"],for_eye["eyePositionCombinedWorld.z"]))
    hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))

    # v_gaze_vec: get difference in hpoo
    v_vX = for_eye["hitPointOnObject_x"].diff().tolist()
    v_vY = for_eye["hitPointOnObject_y"].diff().tolist()
    v_vZ = for_eye["hitPointOnObject_z"].diff().tolist()
    

    # get difference in time:
    ts = pd.DataFrame(time).apply(lambda x: x.diff())[0].tolist()
    v_gaze_vec = list(zip(v_vX, v_vY, v_vZ))

    # gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
    g_vec = list(np.subtract(hpoo, subj))
    gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]
   

    # v_gaze_inplane: is a scalar indicating the velocity in world coordinates at the location that is gazed at orthogonal to the gaze axis.
    # v_gaze_inplane(t) = ||v_gaze_vec(t) - (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))||:
    # z1 = (<v_gaze_vec(t), gaze_vec(t)>)
    z1 = [np.array(v_gaze_vec[t]).dot(np.array(gaze_vec[t]))
            for t in range(len(v_vX))]

    # z = (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))
    z = [z1[t] * np.array(gaze_vec[t]) for t in range(len(v_vX))]

    # ||v_gaze_vec(t) - z||
    v_gaze_inplane = [np.linalg.norm((np.array(v_gaze_vec[t]) - z[t]).tolist())
            for t in range(len(v_gaze_vec))
                     ]
    # w_gaze(t) = arctan2(||subject_vec(t) - hpoo_vec(t)||, v_gaze_inplane)

    # sub_hpoo = ||subject_vec(t) - hpoo_vec(t)||
    sub_hpoo = [np.linalg.norm(np.array(subj[t]) - np.array(hpoo[t]))
            for t in range(len(hpoo))]

    # arctan2(v_gaze_inplane, sub_hpoo)
    w_gaze = np.arctan2(v_gaze_inplane, sub_hpoo).tolist()

    # turn angle of radians into degrees
    #w_gaze = [np.rad2deg(value) for value in w_gaze]
    w_gaze = [(w / ts[idx] * 180 / math.pi) for idx, w in enumerate(w_gaze)]
    # save df
    for_eye["combined_vel"] = w_gaze
   ### 10 second starts here 
    int_len = 10  # number of seconds of the interval

    time = for_eye["timeStampDataPointEnd"].tolist()
    start = []
    end = []
    start_idx = []
    end_idx = []
    for t, ti in enumerate(time):
        if ti == time[0]:
            start.append(ti)
            start_idx.append(t)
        if ti - start[-1] > int_len:
            # if the current timepoint is more than int_len away from start, set it to new start
            start.append(ti)
            start_idx.append(t)
            # and set end to the timepoint before that
            end.append(time[t - 1])
            end_idx.append(t - 1)
    # add the last timepoint to end
    # (there is a very slim chance that the last start and end are the same timepoint --> might cause errors)
    end.append(time[-1])
    end_idx.append(len(time))

    # save it as new df
    int_data = list(zip(start, end, start_idx, end_idx))
    int_data = pd.DataFrame(
        int_data, columns=["start", "end", "start_idx", "end_idx"]
    )

    combined_vel = for_eye["combined_vel"]

    # to shorten the slicing in the next for loop
    start_idx = int_data["start_idx"].tolist()
    end_idx = int_data["end_idx"].tolist()

    # to add the final thresholds to for each segement
    scct = []
    for s, srt in enumerate(int_data["start"].tolist()):
        # get the slice of the combined velocity
        angular_vel = combined_vel[start_idx[s] : end_idx[s]]
        # use the at_mad function to caluclate the threshold
        saccade_th, thres = at_mad(angular_vel)
        if np.isnan(saccade_th):
            scct.append(thres[0])
        else:
            # add it to scct
            scct.append(saccade_th)

    # add it to int_data and save
    int_data["thresh"] = scct
    int_data = pd.DataFrame(int_data)
    time = for_eye.timeStampDataPointEnd.tolist()
    start_idx = int_data["start_idx"].tolist()
    end_idx = int_data["end_idx"].tolist()
    thr = int_data["thresh"].tolist()

    # go through all time intervals
    thresh = [0.0] * len(time)
    for s, srt in enumerate(int_data["start"].tolist()):
        # repeat the threshold as often as the time interval is long
        thresh = (
            thresh[: start_idx[s]]
            + [thr[s]] * len(time[start_idx[s] : end_idx[s]])
            + thresh[end_idx[s]:]
        )

    # add the two lists (ht & et) to for_eye df
    for_eye["thresh"] = thresh

    # save for_eye df
    for_eye = pd.DataFrame(for_eye)
    
    start = int_data["start_idx"].tolist()
    end = int_data["end_idx"].tolist()

    # eye-tracking
    thres = int_data["thresh"].tolist()
    combined_vel = for_eye["combined_vel"].tolist()

    # define list where the fixations will be added too
    is_fix = [np.nan] * len(combined_vel)

    for i in range(len(start)):
        av = combined_vel[start[i] : end[i]]
        # go through combined velocity and save all that are bigger than the threshold
        fix = [ti if ti < thres[i] else np.nan for ti in av]
        is_fix[start[i] : end[i]] = fix

    # save
    for_eye["isFix"] = is_fix

    # save data
    for_eye = pd.DataFrame(for_eye)
    #Everywhere  where there is Nans that is a saccade meaning this are the cells that are really fast 
    
    for_eye.reset_index(inplace=True)
    
    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    time = for_eye.timeStampDataPointEnd.tolist()
    index = for_eye.index.tolist()  # index of df for easier use

    start_time = time[0]  # update for each change
    start_idx = index[0]  # will be updated each event and used to add to the lists

    # to save:
    isFix = []
    combined_vel = []

    # if the first sample does not have any data
    if math.isnan(for_eye.iloc[0]["combined_vel"]) and not math.isnan(
        for_eye.iloc[1]["combined_vel"]
    ):
        start_time = time[1]   # update for each change
        start_idx = index[1]  # will be updated each event and used to add to the lists
        isFix = [np.nan]
        combined_vel = [np.nan]

    # starting with a sacc
    if math.isnan(for_eye.loc[start_idx]["isFix"]):
        event = 0  # == sacc
    # starting with a gaze
    else:
        event = 1  # == gaze

    # go through the list:
    for idx in index[index.index(start_idx) : -1]:
        curr_line = for_eye.loc[idx]
        next_line = for_eye.loc[idx+1]

        # gaze (--> sacc): now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
            if event == 1 and next_line.timeStampDataPointEnd - start_time < min_gaze_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event to small and we are in big saccade, change isFix, change combined_vel
            elif event == 0 and next_line.timeStampDataPointEnd - start_time < min_gaze_dur:
                isFix = isFix + [np.nan] * (idx + 1 - start_idx)
                combined_vel = combined_vel + [np.nan] * (idx + 1 - start_idx)
            # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
            elif next_line.timeStampDataPointEnd - start_time >= min_gaze_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 1  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = for_eye.loc[idx + 1]["timeStampDataPointEnd"]

        # sacc (--> gaze): now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
            if event == 0 and next_line.timeStampDataPointEnd - start_time < min_sacc_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event to small but we are in big gaze, change isFix, change combined_vel
            elif event == 1 and next_line.timeStampDataPointEnd - start_time < min_sacc_dur:
                isFix = (
                    isFix
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # change isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
            elif next_line.timeStampDataPointEnd - start_time >= min_sacc_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 0  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = for_eye.loc[idx + 1]["timeStampDataPointEnd"]

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
                if (
                    event == 1
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small and we are in big saccade, change isFix, change combined_vel
                elif (
                    event == 0
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = isFix + [np.nan] * (idx + 2 - start_idx)
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )
                # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
                elif next_line.timeStampDataPointEnd + 0.011 - start_time >= min_gaze_dur:
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel
            # sacc:
            elif math.isnan(next_line.isFix):
                # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
                if (
                    event == 0
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small but we are in big gaze, change isFix, change combined_vel
                elif (
                    event == 1
                    and next_line.timeStampDataPointEnd + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # change isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
                elif next_line.timeStampDataPointEnd + 0.011 - start_time >= min_sacc_dur:
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel

    # save everything:
    for_eye["isFix"] = isFix
    for_eye["corrected_vel"] = combined_vel
    # save data
    for_eye = pd.DataFrame(for_eye)
    
    time = for_eye.timeStampDataPointEnd.tolist()
    
    ########## EVENTS, LENGTH, AVG DISTANCE, NAME OF OBJECT ##########
    index = for_eye.index.tolist()  # index of df for easier use

    events = [np.nan] * len(
        for_eye
    )  # sacc begin == 1, sacc end == -1; gaze begin == 2, gaze end == -2

    # if the first sample does not have any data
    if math.isnan(for_eye.iloc[0]["combined_vel"]) and not math.isnan(
        for_eye.iloc[1]["combined_vel"]
    ):
        start_idx = index[
            1
        ]  # will be updated each event and used to add to the lists
        events[1] = 2
        length = [np.nan]
        dist = [
            np.nan
        ]  # to save the distance to the hitpoint at each timestamps
        avg_dist = [
            np.nan
        ]  # to save the average distance of collider(s) during event
        names = [np.nan]  # to save the name of the current gaze
    else:
        start_idx = index[
            0
        ]  # will be updated each event and used to add to the lists
        length = []
        dist = []  # to save the distance to the hitpoint at each timestamps
        avg_dist = (
            []
        )  # to save the average distance of collider(s) during event
        names = []  # to save the name of the current gaze
        if math.isnan(for_eye.iloc[index[0]]["combined_vel"]):
            events[0] = 1
        else:
            events[0] = 2

    start_time = for_eye.loc[start_idx]["timeStampDataPointEnd"].tolist()
    # go through the list:
    for idx in index[index.index(start_idx) : -1]:
        curr_line = for_eye.loc[idx]
        next_line = for_eye.loc[idx + 1]

        # distance:
        hpoo = np.array(
            [curr_line.hitPointOnObject_x, curr_line.hitPointOnObject_y, curr_line.hitPointOnObject_z]
        )  # hitpoints on object
        coord_orig = np.array(
            [
                curr_line["eyePositionCombinedWorld.x"],
                curr_line["eyePositionCombinedWorld.y"],
                curr_line["eyePositionCombinedWorld.z"],
            ]
        )  # position of eyes
        dist = dist + [
            np.linalg.norm(hpoo - coord_orig)
        ]  # calculate to distance at this timpoint

        # gaze --> sacc: now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            # get name:
            res = dict(
                Counter(for_eye.loc[start_idx:idx, "Interpolated_collider"].values.tolist())
            )
            names = names + [
                max(res.keys(), key=(lambda new_k: res[new_k]))
            ] * (idx + 1 - start_idx)
            # length, distance, events
            length = length + [curr_line.timeStampDataPointEnd - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event
            events[index.index(idx)] = -2  # end of gaze
            events[index.index(idx) + 1] = 1  # beginning of sacc
            # new idx
            start_time = curr_line.timeStampDataPointEnd
            start_idx = idx + 1

        # sacc --> gaze: now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            # get name:
            res = dict(
                Counter(for_eye.loc[start_idx:idx, "Interpolated_collider"].values.tolist())
            )
            names = names + [
                max(res.keys(), key=(lambda new_k: res[new_k]))
            ] * (idx + 1 - start_idx)
            # length, distance, events
            length = length + [curr_line.timeStampDataPointEnd - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event
            events[index.index(idx)] = -1  # end of sacc
            events[index.index(idx) + 1] = 2  # beginning of gaze
            # new idx
            start_time = curr_line.timeStampDataPointEnd
            start_idx = idx + 1

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                events[-1] = -2  # end of gaze
            # sacc:
            elif math.isnan(next_line.isFix):
                events[-1] = -1  # end of sacc
            length = length + [next_line.timeStampDataPointEnd - start_time] * (
                idx + 2 - start_idx
            )  # length of event
            # distance
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 2 - start_idx
            )  # average distance to collider(s) during event
            hpoo = np.array(
                [curr_line.hitPointOnObject_x, curr_line.hitPointOnObject_y, curr_line.hitPointOnObject_z]
            )  # hitpoints on object
            coord_orig = np.array(
                [
                    curr_line["eyePositionCombinedWorld.x"],
                    curr_line["eyePositionCombinedWorld.y"],
                    curr_line["eyePositionCombinedWorld.z"],
                ]
            )  # position of eyes
            dist = dist + [
                np.linalg.norm(hpoo - coord_orig)
            ]  # calculate to distance at this timpoint
            # names
            res = dict(
                Counter(for_eye.loc[start_idx:, "Interpolated_collider"].values.tolist())
            )
            names = names + [
                max(res.keys(), key=(lambda new_k: res[new_k]))
            ] * (idx + 2 - start_idx)
    # save everything:
    for_eye["events"] = events
    for_eye["length"] = length
    for_eye["distance"] = dist
    for_eye["avg_dist"] = avg_dist
    for_eye["names"] = names
    # display(for_eye[['time','isFix','events','hon_all','names']])
    # save data
    for_eye = pd.DataFrame(for_eye)
    # Change average distance to correct for the potential of other events 
    # so distance and avg_dist

    # total lists:
    all_dist = []
    avg_dist = []

    # updated after each gaze
    dist = []
    hon_pos = []
    dur_gaze = False

    # during event:
    # go through the list:      
    for g,gz in enumerate(for_eye['events']):
        curr_line = for_eye.loc[g]
        if gz == 2.0 or gz == 1.0:
            dur_gaze = True
            # get the gazed at object
            curr_gaze = curr_line.names
        # if you are currently in a gaze:
        if dur_gaze:
            # if you are currently having the correct element, add the position
            if curr_line.Interpolated_collider == curr_gaze:    
                hon_pos = hon_pos + [[curr_line.hitPointOnObject_x,
                    curr_line.hitPointOnObject_y,
                    curr_line.hitPointOnObject_z,]]
            dist = dist + [np.array([curr_line["eyePositionCombinedWorld.x"],
                curr_line["eyePositionCombinedWorld.y"],
                curr_line["eyePositionCombinedWorld.z"],])]

        # once the gaze is over, take the avg_dist
        if gz == -2.0 or gz == -1.0:
            hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
            # calculate to distance at this timpoint
            dist = [np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))]
            all_dist = all_dist + dist
            # average distance during the gaze event
            avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

            # reset everything:
            dist = []
            hon_pos = []
            dur_gaze = False

        # if there are parts that are neither gaze nor saccade:
        if (not dur_gaze) and (gz not in [2.0,1.0]) and (len(all_dist) + len(dist) != g + 1):
            all_dist = all_dist + [np.nan]
            avg_dist = avg_dist + [np.nan]

        if len(all_dist) + len(dist) != g + 1:
            display(g)

    if dur_gaze:
        hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
        # calculate to distance at this timpoint
        dist = [np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))]
        all_dist = all_dist + dist
        # average distance during the gaze event
        avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

    # add them to for_eye
    for_eye['distance'] = all_dist
    for_eye['avg_dist'] = avg_dist
    
    for_eye["names"] =  for_eye.names.fillna(method='ffill').fillna(method='bfill')
    for_eye['Collider_CategoricalN'] = for_eye['names'].apply(lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))
    

    for_eye = pd.DataFrame(for_eye)
    
    # Define the condition and the string to add
    Mask_1f = ((for_eye['Collider_CategoricalN'] == "Active_Agent") & (for_eye['Face_Hits'] == "Face")) | ((for_eye['Collider_CategoricalN'] == "Passive_Agent") & (for_eye['Face_Hits'] == "Face"))
    Mask_2f  = ((for_eye['Collider_Categorical'] == "Active_Agent") | (for_eye['Collider_Categorical'] == "Passive_Agent")) & (for_eye['Face_Hits'] == "Face")    
    string_to_add = "_Face"
    # Use the loc method to index the rows where the condition is met
    for_eye.loc[Mask_1f, 'Collider_CategoricalN'] = for_eye.loc[Mask_1f, 'Collider_CategoricalN'] + string_to_add
    for_eye.loc[Mask_2f, 'Collider_Categorical'] = for_eye.loc[Mask_2f, 'Collider_Categorical'] + string_to_add
    for_eye.to_csv(f"/Volumes/SSD/00_Data_Processing/Pre_processed/05_Debbies_gaze/{filename[-10:-4]}.csv", index=True)
    print(filename[-10:-4])

    

ERROR! Session/line number was not unique in database. History logging moved to new session 1249


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-5e621d054a72>", line 14, in <module>
    One_participant = pd.read_csv(filename)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 468, in _read
    return parser.read(nrows)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 1057, in read
    index, columns, col_dict = self._engine.read(nrows)
  File "/Users/tracysanchezpacheco/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 2061, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parse

TypeError: object of type 'NoneType' has no len()

In [ ]:
path = "/Volumes/SSD/00_Data_Processing/Pre_processed/04_Interpolated" 

# csv files in the path
files = glob.glob(path + "/*.csv")
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    One_participant = pd.read_csv(filename)
    # Here we identify the shifts on collider name
    One_participant['Collider_shift'] = One_participant['Interpolated_collider'].shift(1) != One_participant['Interpolated_collider']
    # Create calculate a cumulative sum of the collider changes
    One_participant['counter'] = (One_participant['Collider_shift'] == True).cumsum()
    # Shift the counter column by one row to align it with the correct row
    One_participant['counter'] = One_participant['counter'].shift(1).fillna(0)
    One_participantC = One_participant.copy()
    #Create subset that only has the rows with shifts in colliders 
    One_participant_true = One_participant[One_participant['Collider_shift'] == True].reset_index().copy()
    # Since shift converts index into float we change it back into int so that it can be read as index
    One_participant_true["index_shift"] = One_participant_true["index"].shift(-1).astype('Int64')
    # Calculate the difference in time between each shift
    One_participant_true["Time_diff"] = One_participant_true.timeStampDataPointEnd.diff(1).shift(-1)
    One_participant_true.dropna(inplace=True)
    #### Create the gaze column 
    One_participant["Time_of_Gaze"] = np.nan
    low = One_participant_true["index"].to_list()
    up = One_participant_true["index_shift"].to_list()
    time =  One_participant_true["Time_diff"].to_list()
    ranges = list(zip(low, up))
    for i, (lower, upper) in enumerate(ranges):
        One_participant.loc[lower:upper,"Time_of_Gaze"]  = time[i]
    One_participant["Gaze"] = np.where(One_participant["Time_of_Gaze"] > .250, "Gaze", "Movement")
    One_participant_true["Gaze"] = np.where(One_participant_true["Time_diff"] > .250, "Gaze", "Movement")
    low = []
    up = []
    
    ######## Debbies Algorithm ########
    
    for_eye = One_participant.copy()
   
    time = for_eye["timeStampDataPointEnd"].tolist()
    # get individual coordinates
    subj = list(zip( for_eye["eyePositionCombinedWorld.x"],for_eye["eyePositionCombinedWorld.y"],for_eye["eyePositionCombinedWorld.z"]))
    hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))

    # v_gaze_vec: get difference in hpoo
    v_vX = for_eye["hitPointOnObject_x"].diff().tolist()
    v_vY = for_eye["hitPointOnObject_y"].diff().tolist()
    v_vZ = for_eye["hitPointOnObject_z"].diff().tolist()
    

    # get difference in time:
    ts = pd.DataFrame(time).apply(lambda x: x.diff())[0].tolist()
    v_gaze_vec = list(zip(v_vX, v_vY, v_vZ))

    # gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
    g_vec = [np.array(hpoo[v] - np.array(subj[v])) for v in range(len(subj))]
    gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]

In [ ]:
Remember the rolling window thing 

df["diff"] = df["value"].rolling(window=2).apply(lambda x: x[1] - x[0])

# Debbies plot

In [ ]:
for_eye = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/03_Debbies_gaze/One_participant_WDC.csv", index_col="timeStampDataPointEnd")

window_lower = for_eye.index.tolist()[2070]
window_upper = for_eye.index.tolist()[2270]

titel = "Hit Points of Gazes"
            

# get time:
ts = for_eye.index.tolist()  # to make it easier
time = ts[
    ts.index(
        list(filter(lambda i: i > window_lower, ts))[0]
    ) : ts.index(list(filter(lambda i: i < window_upper, ts))[-1])
    + 1
]  # get all timestamps in the important time window

# get shorter df:
for_eye = for_eye.iloc[ts.index(time[0]) : (ts.index(time[-1]) + 1)]


# hon: for showing lines in plot
hon = for_eye["Interpolated_collider"].tolist()
new_col = [
    hon[n] if hon[n] != hon[n - 1] and not pd.isnull(hon[n]) else np.nan
    for n in range(len(hon))
]

hon_ts = [
    ti for cnt, ti in enumerate(time) if isinstance(new_col[cnt], str)
]  # timestamps

# get gazes:
gaze = for_eye[~for_eye["isFix"].isnull()]
gaze = gaze.rename({'hitPointOnObject_x': 'xgaze', 'hitPointOnObject_y': 'ygaze', 'hitPointOnObject_z': 'zgaze'}, axis=1)

sacc = for_eye[~for_eye.index.isin(gaze.index)]
sacc = sacc.rename({'hitPointOnObject_x': 'xsacc', 'hitPointOnObject_y': 'ysacc', 'hitPointOnObject_z': 'zsacc'}, axis=1)


# plot it:
sns.set(rc={"figure.figsize": (17, 9)})
sns.set_style(
    "white"
)  # styledict, or one of {darkgrid, whitegrid, dark, white, ticks}

f, (axis) = plt.subplots(2, 1)


for x, xc in enumerate(hon_ts):
    if not np.isnan(xc):
        axis[0].axvline(
            x=xc, color="#987284", alpha=0.2, label="_Hidden label"
        )

color_gaze = {
    "xgaze": "#5FAD56",
    "ygaze": "#27408B",
    "zgaze": "#4C86A8",
}
color_sacc = {
    "xsacc": "#BA1200",
    "ysacc": "#CD96CD",
    "zsacc": "#F0A202",
}

gaze[["xgaze", "ygaze", "zgaze"]].plot(
    color=[
        color_gaze.get(x, "#333333")
        for x in gaze[["xgaze", "ygaze", "zgaze"]]
    ],
    ax=axis[0],
    marker="o",
    ls="",
)

sacc[["xsacc", "ysacc", "zsacc"]].plot(
    color=[
        color_sacc.get(x, "#333333")
        for x in sacc[["xsacc", "ysacc", "zsacc"]]
    ],
    ax=axis[0],
    marker="o",
    ls="",
)

axis[0].set_title(
    titel,
    fontsize=22,
    fontweight='bold',
)
axis[0].legend(loc="upper right", fontsize=18)
axis[0].xaxis.label.set_visible(False)
axis[0].set_ylabel("coordinates", fontsize=20)
axis[0].yaxis.set_tick_params(labelsize = 14) # change tick size
axis[0].xaxis.set_tick_params(labelsize = 14) 



axis[1].plot(time, for_eye["combined_vel"].tolist(), "g", label = "combined_vel")
axis[1].plot(time, for_eye["thresh"].tolist(), "r", label = "threshold")

#axis[3].plot(time, long_events_mad, "k")
## axis[2].plot(time, blinks, "b")  # blinks
axis[1].set_ylim(0, 600)
axis[1].set_title(
        "Velocities",
        fontsize=22,
        fontweight='bold',
    )
axis[1].legend(loc="upper right", fontsize=18)
#plt.xticks(fontsize=14)
#ax.set_xticklabels(time,fontsize=20)
#plt.suptitle(uid, fontsize=20)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
axis[1].set_xlabel("time (sec)", fontsize=20)
axis[1].set_ylabel("veloctiy", fontsize=20)
axis[1].yaxis.set_tick_params(labelsize = 14) # change tick size
axis[1].xaxis.set_tick_params(labelsize = 14) 

plt.suptitle(titel, fontsize=24)


In [ ]:
for_eye